## Functions

In [233]:
def yes_no_every_second(start, finish): #creates an  entry for every second of the day between 5am (18000) and 8pm (72000), 1 for a yes and 0 for a no 
    i = 18000  # ie starts at 5am
    yes_list = []
    while i < 72000: # ends the time span at 8pm and prevents traceback error for behaviours that are not recorded in a given day
        for s in range(0, len(start)): # len(start) is the number of instances of the behaviour recorded that day
            if  i == int(start[s]) or i == int(finish[s]): # this deals with the start and finish time of a behaviour instance
                yes_list.append(1)
                #print 'yes', i
            elif i > int(start[s]) and i < int(finish[s]): #deals with seconds in between a start and finish time
                yes_list.append(1)
                #print 'yes', i
            elif s-1 >= 0 and i > int(finish[s-1]) and i < int(start[s]):
                yes_list.append(0)
                #print 'no', i
            elif s == (len(start) -1) and i > int(finish[s]):
                yes_list.append(0)
                #print 'no', i
            elif s == 0 and i < int(start[s]):
                yes_list.append(0)
                #print 'no', i
                
        i = i + 1
    #print len(yes_list)
    return yes_list

## Import Packages and set working directory

In [234]:
import os
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
os.chdir("C:/Users/bethc/Desktop/DBS/DBS_Project/Data")


## Read in the data and name the columns

In [235]:
#read in required columns from csv file and name columns
col_names =['DayNo', 'Date', 'Behaviour', 'StartTime', 'FinishTime']
zoo = pd.read_csv("master_data_for_python_test.csv", header = None, usecols=[0, 1,3,4,5], names=col_names, parse_dates = ['Date'])
zoo = zoo.iloc[0:773]

## Check everything is as it should be

In [236]:
zoo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 5 columns):
DayNo         773 non-null float64
Date          773 non-null datetime64[ns]
Behaviour     773 non-null object
StartTime     773 non-null object
FinishTime    773 non-null object
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 30.3+ KB


In [237]:
#take a look to make sure everything loaded correctly
zoo.head()

,DayNo,Date,Behaviour,StartTime,FinishTime
0,1.0,2017-08-14,LY,05:00:00,06:09:03
1,1.0,2017-08-14,FO,06:09:03,06:09:52
2,1.0,2017-08-14,SC,06:09:52,06:10:08
3,1.0,2017-08-14,FO,06:10:08,06:11:21
4,1.0,2017-08-14,AR,06:11:21,06:15:46


## Convert Start and Finish times to seconds

In [238]:
#convert start and finish time to seconds
zoo['start_time'] = pd.to_timedelta(zoo.StartTime)
zoo['start_time'] = zoo['start_time'].dt.total_seconds().astype(int)


zoo['finish_time'] = pd.to_timedelta(zoo.FinishTime)
zoo['finish_time'] = zoo['finish_time'].dt.total_seconds().astype(int)

zoo.head()


,DayNo,Date,Behaviour,StartTime,FinishTime,start_time,finish_time
0,1.0,2017-08-14,LY,05:00:00,06:09:03,18000,22143
1,1.0,2017-08-14,FO,06:09:03,06:09:52,22143,22192
2,1.0,2017-08-14,SC,06:09:52,06:10:08,22192,22208
3,1.0,2017-08-14,FO,06:10:08,06:11:21,22208,22281
4,1.0,2017-08-14,AR,06:11:21,06:15:46,22281,22546


In [239]:
#create subset data frame with just 1 day and 1 behaviour
zoo_SW_1 = zoo[(zoo.Behaviour == 'SW') & (zoo.DayNo == 1.0)]
zoo_SW_1.head()

,DayNo,Date,Behaviour,StartTime,FinishTime,start_time,finish_time
23,1.0,2017-08-14,SW,07:00:14,07:00:47,25214,25247
25,1.0,2017-08-14,SW,07:00:58,07:04:55,25258,25495
29,1.0,2017-08-14,SW,07:06:17,07:07:07,25577,25627
32,1.0,2017-08-14,SW,07:08:34,07:37:11,25714,27431
34,1.0,2017-08-14,SW,07:37:30,07:42:27,27450,27747


In [240]:
#create new data frame
day1_SW_dict = {'DayNo': 1, 'Behaviour': 'SW', 'time' : pd.timedelta_range(start='05:00:00', end='19:59:59', freq='1S')}
day1_SW = pd.DataFrame(day1_SW_dict)
day1_SW = day1_SW.set_index('time')
day1_SW.head()

,Behaviour,DayNo
time,,
05:00:00,SW,1
05:00:01,SW,1
05:00:02,SW,1
05:00:03,SW,1
05:00:04,SW,1


In [241]:
#create list of start times
start = zoo_SW_1['start_time'].tolist()
finish = zoo_SW_1['finish_time'].tolist()


In [242]:
#create list of yes and no entries
day1_SW['yes_no'] = yes_no_every_second(start, finish)


In [243]:
#convert to %TSB in 15 minute intervals
TSB = (day1_SW['yes_no']).resample('15min').sum().apply(lambda x: (float(x)/900)*100)
